In [1]:
import pandas as pd
import json

In [9]:
df_train = pd.read_parquet('scotus_eyecite_train.pqt')

In [10]:

df_train['spans']

0       [{"volume": "329", "reporter": "U.S.", "page":...
1       [{"volume": "329", "reporter": "U.S.", "page":...
2       [{"volume": "329", "reporter": "U.S.", "page":...
3       [{"volume": "329", "reporter": "U.S.", "page":...
4       [{"volume": "329", "reporter": "U.S.", "page":...
                              ...                        
4995    [{"volume": "459", "reporter": "U.S.", "page":...
4996    [{"volume": "459", "reporter": "U.S.", "page":...
4997    [{"volume": "459", "reporter": "U.S.", "page":...
4998    [{"volume": "459", "reporter": "U.S.", "page":...
4999    [{"volume": "459", "reporter": "U.S.", "page":...
Name: spans, Length: 5000, dtype: object

In [36]:

def check_groups(row):
    groups = json.loads(row['groups'])
    spans = json.loads(row['spans'])
    def check_span(start,end,groups):
        for group in groups:
            for item in group:
                if item['start'] == start and item['end'] == end:
                    return True
        return False
    for span in spans:
        start = span['start']
        end = span['end']
        assert check_span(start, end,groups),span


def check_groups_inside_allowed(row):
    groups = json.loads(row['groups'])
    spans = json.loads(row['spans'])
    def check_span(start,end,groups):
        for group in groups:
            for item in group:
                if item['start'] <= start and item['end'] >= end:
                    return True
        return False
    for span in spans:
        start = span['start']
        end = span['end']
        assert check_span(start, end,groups),span



In [37]:
spans = json.loads(df_train.iloc[0]['spans'])
groups = json.loads(df_train.iloc[0]['groups'])
text = df_train.iloc[0]['text']


In [42]:
import eyecite
import eyecite.models
citations = eyecite.get_citations(text)

In [43]:
len(citations)

65

In [49]:
unknown_citations = [c for c in citations if isinstance(c,eyecite.models.UnknownCitation)]
known_citations = [c for c in citations if not isinstance(c,eyecite.models.UnknownCitation)]


In [50]:
len(unknown_citations)

26

In [51]:
spans2 = sum(groups,[])
len(spans2),len(unknown_citations),len(known_citations)
           
    



(39, 26, 39)

In [31]:
def inside(span,item):
    return span['start'] >= item['start'] and span['end'] <= item['end']

for span in spans:
    found = False
    for group in groups:
        for item in group:
            if item == span:
                print(span,'ok')
                found = True
            elif inside(span,item):
                print(span,'inside')
                found = True
            elif inside(item,span):
                print(span,"outside")
    if not found:
        print(span,'missing')

{'volume': '329', 'reporter': 'U.S.', 'page': '29', 'text': '329 U.S. 29', 'start': 0, 'end': 11} ok
{'volume': '67', 'reporter': 'S.Ct.', 'page': '1', 'text': '67 S.Ct. 1', 'start': 12, 'end': 22} ok
{'volume': '91', 'reporter': 'L.Ed.', 'page': '22', 'text': '91 L.Ed. 22', 'start': 23, 'end': 34} ok
{'volume': '329', 'reporter': 'U.S.', 'page': '831', 'text': '329 U.S. 831', 'start': 172, 'end': 184} ok
{'volume': '67', 'reporter': 'S.Ct.', 'page': '363', 'text': '67 S.Ct. 363', 'start': 186, 'end': 198} ok
{'text': '§', 'start': 603, 'end': 604} missing
{'text': '§', 'start': 1054, 'end': 1055} missing
{'title': '28', 'reporter': 'U.S.C.', 'section': '345', 'text': '28 U.S.C. § 345', 'start': 1061, 'end': 1076} ok
{'title': '28', 'reporter': 'U.S.C.A.', 'section': '345', 'text': '28 U.S.C.A. § 345', 'start': 1078, 'end': 1095} ok
{'volume': '234', 'reporter': 'U.S.', 'page': '548', 'text': '234 U.S. 548', 'start': 2989, 'end': 3001} ok
{'volume': '34', 'reporter': 'S.Ct.', 'page': '

NameError: name 'small' is not defined

In [13]:
resolved

defaultdict(list,
            {Resource(citation=FullJournalCitation('25 Texas L.Rev. 199', groups={'volume': '25', 'reporter': 'Texas L.Rev.', 'page': '199'}, metadata=ResourceCitation.Metadata(parenthetical=None, pin_cite=None, year=None))): [FullJournalCitation('25 Texas L.Rev. 199', groups={'volume': '25', 'reporter': 'Texas L.Rev.', 'page': '199'}, metadata=ResourceCitation.Metadata(parenthetical=None, pin_cite=None, year=None))]})

In [30]:
cits[0].span()

(12, 22)

{'keys': [(0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11),
  (0, 11)]}

In [68]:
resolved

0    {Resource(citation=FullCaseCitation('329 U.S. ...
1    {Resource(citation=FullCaseCitation('329 U.S. ...
2    {Resource(citation=FullCaseCitation('329 U.S. ...
3    {Resource(citation=FullCaseCitation('329 U.S. ...
4    {Resource(citation=FullCaseCitation('329 U.S. ...
Name: text, dtype: object

In [70]:
list(resolved.items())[-1]

(4,
 defaultdict(list,
             {Resource(citation=FullCaseCitation('329 U.S. 90', groups={'volume': '329', 'reporter': 'U.S.', 'page': '90'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('329 U.S. 90', groups={'volume': '329', 'reporter': 'U.S.', 'page': '90'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))],
              Resource(citation=FullCaseCitation('67 S.Ct. 133', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '133'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('67 S.Ct. 133', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '133'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=

defaultdict(list,
            {Resource(citation=FullCaseCitation('329 U.S. 29', groups={'volume': '329', 'reporter': 'U.S.', 'page': '29'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('329 U.S. 29', groups={'volume': '329', 'reporter': 'U.S.', 'page': '29'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))],
             Resource(citation=FullCaseCitation('67 S.Ct. 1', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '1'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('67 S.Ct. 1', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '1'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant

In [76]:
def span_pairs(item):
    match item:
        case dict():
            for k,v in item.items():
                source = span(k)
                for item in v:
                    yield source,span(item)
                    
        case _:
            yield item
        
        
def span(item):
    match item:
        case eyecite.models.Resource():
            return span(item.citation)
        case eyecite.models.CitationBase():
            return item.span()

In [88]:
def get_links(text):
    citations = eyecite.get_citations(text)
    resolved = eyecite.resolve_citations(citations)
    links = [dict(target_text=text[target[0]:target[1]],
              target_span=target,
              source_span=text[source[0]:source[1]],
              source=source) for target,source in span_pairs(resolved) if target != source]
    return links


In [93]:
print(json.dumps(get_links(df_train['text'][2]),indent=4))

[
    {
        "target_text": "322 U.S. 369",
        "target_span": [
            1377,
            1389
        ],
        "source_span": "322 U.S. 369",
        "source": [
            25595,
            25607
        ]
    },
    {
        "target_text": "64 S.Ct. 1037",
        "target_span": [
            1391,
            1404
        ],
        "source_span": "64 S.Ct. 1037",
        "source": [
            25609,
            25622
        ]
    },
    {
        "target_text": "88 L.Ed. 1331",
        "target_span": [
            1406,
            1419
        ],
        "source_span": "88 L.Ed. 1331",
        "source": [
            3328,
            3341
        ]
    },
    {
        "target_text": "88 L.Ed. 1331",
        "target_span": [
            1406,
            1419
        ],
        "source_span": "88 L.Ed. 1331",
        "source": [
            7523,
            7536
        ]
    },
    {
        "target_text": "88 L.Ed. 1331",
        "target_span": [
         